In [1]:
import sys
import os 
import numpy as np
import time
import tensorflow as tf
from PIL import Image
from tensorflow.python.framework import graph_util  
from tensorflow.python.platform import gfile  
#myself
import src.make_siameseFC as siam
from src.train_siam_net import train_siam_net
from src.region_to_bbox import region_to_bbox
from src.parse_arguments import parse_arguments

_conv_w_sz=np.array([11,5,3,3,3]) #the map size of filter(weight of conv net)
_conv_w_in_c=np.array([3,96,256,384,384])# the input channle number of filter
_conv_w_out=np.array([96,256,384,384,256])# the output number of feature map(the number of filter kernel)
_nums_layers=5

def main():
    #avoid printing TF debugging information(only show error log)
    os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
    hp,evaluation,run,env,design=parse_arguments()
    #build TF graph in siamese once for all
    #siam.init_create_net()
    filename,siam_net_z,loss,train_op=siam.make_siameseFC(env,design,hp)
    #read mat file from net_path
    params_names_list,params_values_list=_import_from_matconvent(net_path)
    conv_W=[1,_num_layers]
    for i in range(_nums_layers):
        conv_W_name=_find_params('conv'+str(i+1)+'f',params_names_list)[0]
        conv_b_name=_find_params('conv'+str(i+1)+'b',params_names_list)[0]
        conv_W[i]=params_values_list[params_names_list.index(conv_W_name)]
        conv_b[i]=params_values_list[params_names_list.index(conv_b_name)]
    
    #iterate through all videos of evaluation.dataset
    if evaluation.video == 'all':
        #the path of folder of all videos
        train_data_folder=os.path.join(env.root_train_dataset,evaluation.dataset)
        print(train_data_folder)
        videos_list=[v for v in os.listdir(train_data_folder)]
        videos_list.sort()
        num_v=len(videos_list)
        print(num_v)
        for i in range(num_v):
            gt,frame_name_list,_,_ = _init_train_video(env,evaluation,videos_list[i])
            start_frame=evaluation.start_frame
            gt_=gt[start_frame:]
            frame_name_list_=frame_name_list[start_frame:]
            num_frames=np.size(frame_name_list_)
            train_siam_net(design,hp,frame_name_list,num_frames,gt,filename,conv_W,conv_b,siam_net_z,loss,train_op)
        
    else:
        gt,frame_name_list,_,_ = _init_train_video(env,evaluation,evaluation.video)
        start_frame=evaluation.start_frame
        gt_=gt[start_frame:]
        frame_name_list_=frame_name_list[start_frame:]
        num_frames=np.size(frame_name_list_)
       
        train_siam_net(design,hp,frame_name_list,num_frames,gt,filename,conv_W,conv_b,siam_net_z,loss,train_op)
            
    #write_file_param(design,env,evaluation)
    
def _init_train_video(env,evaluation,video):
     #-------------------------------------------------------------------------
    #function//init info of a train video sequence
    #-------------------------------------------------------------------------
    #the path of train_data folder
    train_data_folder=os.path.join(env.root_train_dataset,evaluation.dataset,video)
    print(train_data_folder)
    #os.listdir():show the file list of the folder
    frame_name_list=[f for f in os.listdir(train_data_folder) if f.endswith("."+env.image_type)]
    frame_name_list=[os.path.join(train_data_folder,'')+s for s in frame_name_list]
    frame_name_list.sort()
    
    #get the info of first frame
    with Image.open(frame_name_list[0]) as img:
        frame_sz=np.asarray(img.size)
        ##????????????????????????????????????????????
        frame_sz[1],frame_sz[0]=frame_sz[0],frame_sz[1]
        
        #read the initialization from ground truth(init the template_z)
    gt_file=os.path.join(train_data_folder,evaluation.gt_name)
    gt=np.genfromtxt(gt_file,delimiter=evaluation.gt_delimiter)
    num_frame=len(frame_name_list)
    print(num_frame)
    print(len(gt))
    assert num_frame == len(gt),'number of frame and number of gt should be the same'
    
    return gt,frame_name_list,frame_sz,num_frame

if __name__=='__main__':
    sys.exit(main())

Layer conv1
Tensor("conv1/Relu:0", shape=(1, 123, 123, 96), dtype=float32)
Tensor("conv1/Relu_1:0", shape=(1, 59, 59, 96), dtype=float32)
_pool_stride
Tensor("conv1/pool1:0", shape=(1, 61, 61, 96), dtype=float32)
Tensor("conv1/pool1_1:0", shape=(1, 29, 29, 96), dtype=float32)
Layer conv2
Tensor("conv2/Relu:0", shape=(1, 57, 57, 256), dtype=float32)
Tensor("conv2/Relu_1:0", shape=(1, 25, 25, 256), dtype=float32)
_pool_stride
Tensor("conv2/pool2:0", shape=(1, 28, 28, 256), dtype=float32)
Tensor("conv2/pool2_1:0", shape=(1, 12, 12, 256), dtype=float32)
Layer conv3
Tensor("conv3/Relu:0", shape=(1, 26, 26, 384), dtype=float32)
Tensor("conv3/Relu_1:0", shape=(1, 10, 10, 384), dtype=float32)
Tensor("conv3/Relu:0", shape=(1, 26, 26, 384), dtype=float32)
Tensor("conv3/Relu_1:0", shape=(1, 10, 10, 384), dtype=float32)
Layer conv4
Tensor("conv4/Relu:0", shape=(1, 24, 24, 384), dtype=float32)
Tensor("conv4/Relu_1:0", shape=(1, 8, 8, 384), dtype=float32)
Tensor("conv4/Relu:0", shape=(1, 24, 24, 384

SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
